In [1]:
# If you haven't installed mlxtend yet for ipython
## UNCOMENT IF YOU HAVE ANACONDA or DOCKER
# !conda install -c rasbt mlxtend -qy
### UNCOMENT IF YOU PIP INSTALLED EVERYTHING
# !pip install mlxtend

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.evaluate import plot_decision_regions

# Perceptron 
Implement the training portion of the SVM algorithm. Look to [slide 7 from the presentation](https://docs.google.com/a/berkeley.edu/presentation/d/1jaVNN3u7iyN3RGxAw9_61JH4qUio6moCxk1Id1jzYHY/edit?usp=sharinghttps://docs.google.com/a/berkeley.edu/presentation/d/1jaVNN3u7iyN3RGxAw9_61JH4qUio6moCxk1Id1jzYHY/edit?usp=sharing) on the mathematics of training a perceptron.

In [3]:
class Perceptron(object):

    def __init__(self, eta=0.01, epochs=50):
        self.eta = eta
        self.epochs = epochs

    def train(self, X, y):
        # Initialize to random - original notebook mistakenly put np.zeros which only worked with bad implementation
        self.w_ = np.random.randn(X.shape[1])
        self.errors_ = []

        for _ in range(self.epochs):
            # initialize inner product matrix with 0s
            Z = np.zeros(X.shape[0])
            for i in range(X.shape[0]):
                # take the inner product of feature vector i and the weight vector
                Z[i] = self.predict(X[i])
            # return the indexes of datapoints where the perceptron misclassified a point
            V_index = np.where(Z * y < 0)
            
            # calculate gradient of the weight vector w.r.t. the cost fn using only datapoints
            grad = np.dot(y[V_index], X[V_index])

            # adjust the weight vector by the gradient 
            self.w_ = self.w_ + self.eta * grad
            
            # error for epoch is equal to the number of misclassified points
            error = np.size(V_index)
            self.errors_.append(error)
        return self

    def net_input(self, X):
        return np.dot(X, self.w_)

    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

## Iris dataset
Dataset of three different Iris flower species and their respective features. [Source link](https://archive.ics.uci.edu/ml/datasets/Iris)

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

# setosa and versicolor
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', -1, 1)

# sepal length and petal length
X = df.iloc[0:100, [0,2]].values

In [ ]:
ppn = Perceptron(epochs=10, eta=.01)

ppn.train(X, y)
print('Weights: %s' % ppn.w_)
plot_decision_regions(X, y, clf=ppn)
plt.title('Perceptron')
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.show()

plt.plot(range(1, len(ppn.errors_)+1), ppn.errors_, marker='o')
plt.xlabel('Iterations')
plt.ylabel('Missclassifications')
plt.show()

Weights: [-1.72916807  3.24234768]


/Users/philkuz/.virtualenvs/jupyter/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Note that importing this function from mlxtend.evaluate has been deprecated and will not longer be supported in mlxtend 0.6. Please use`from mlxtend.plotting import plot_decision_regions` instead.


# SVM
This example is intended more as a demonstration of the sklearn api rather than a challenge to implement. SVMs are non-trivial to implement so we've omitted that here. If you're interested, [this article](http://tullo.ch/articles/svm-py/) covers an implementation using numpy.

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.evaluate import plot_decision_regions
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

# setosa and versicolor
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-setosa', -1, 1)

# sepal length and petal length
X = df.iloc[0:100, [0,2]].values

## Hard Margin SVM

This calls the SVC class. This fits the standard sklearn api where each model has a `fit()` method which trains the model, and likewise has a `predict()` method that allows you to label new data. This particular implementation is _technically_ a soft-margin SVM, however, as the `C` parameter approaches infinity (and $10^{100}$ computationally is that case), the SVM becomes hard margin.

In [ ]:
hard_svm = SVC(C=1e100,kernel='linear')
hard_svm.fit(X, y)

In [ ]:
def plot_svm(clf, X, Y, title='SVM'):
    # get the separating hyperplane
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(4, 8)
    yy = a * xx - (clf.intercept_[0]) / w[1]

    # plot the parallels to the separating hyperplane that pass through the
    # support vectors
    b = clf.support_vectors_[0]
    yy_down = a * xx + (b[1] - a * b[0])
    b = clf.support_vectors_[-1]
    yy_up = a * xx + (b[1] - a * b[0])

    # plot the line, the points, and the nearest vectors to the plane
    plt.plot(xx, yy, 'k-')
    plt.plot(xx, yy_down, 'k--')
    plt.plot(xx, yy_up, 'k--')

    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
                s=80, facecolors='none')
    plt.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.Paired)

    print('Support Vectors: \n%s' % clf.support_vectors_)

    plt.title(title)
    plt.xlabel('sepal length [cm]')
    plt.ylabel('petal length [cm]')
    plt.axis('tight')
    plt.show()

In [ ]:
plot_svm(hard_svm, X, y)

## Soft-Margin SVM
Now here's an example of a soft-margin SVM. The `C` parameter allows for slack variables. `C=1` is in fact the default argument, so you can omit it if you'd like. However, we've included it for your sake.

As an exercise, you should experiment with different values of `C` and observe how that changes the result. 

Think of  a method for trying out a wide array of different values of `C`. This is part of an important tool in machine learning called [hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization), where `C` is the hyperparamater of the SVM.

In [ ]:
np.unique(df[4])

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

# setosa and versicolor
y = df.iloc[0:100, 4].values
y = np.where(y == 'Iris-versicolor', -1, 1)

# sepal length and petal length
X = df.iloc[0:100, [0,2]].values

In [ ]:
soft_svm = SVC(C=1,kernel='linear')
soft_svm.fit(X, y)

In [ ]:
plot_svm(soft_svm, X, y)